# CS 344 Homework #3
#### Enoch Mwesigwa
#### Keith Vander Linden



## Exercise 1


Gain(Price) = Entropy(WillWait) - Remainder(price)

<br/>
<br/>
Entropy(WillWait) = P(yes) * lg(P(yes)) + P(no) * lg(P(no))
                 <br/> = (.5) * lg(.5) + (.5)log(.5)
                 <br/> = 1
<br/>
<br/>
Remainder(price) = (3 + 4)/(6 + 6) * E(3/7, 4/7) + (2)/(6 + 6) * E(2/2, 0/2) + (1 + 2)/(6 + 6) * E(1/3, 2/3)

<br/>
= (7/12) * -[ (3/7) * lg(3/7) + (4/7) * lg(4/7)] 
<br/> + (2/12) * -[ (2/2) * lg(2/2) + 0 ] 
<br/> + (3/12) * -[ (1/3) * lg(1/3) + (2/3) * lg(2/3) ]


<br/><br/>
Gain(price) = Entropy(WillWait) - Remainder(price)
<br/>
= 1 - 0.804
<br/>
= 0.196


Since the gain for price is lower than the gain for patrons, patron's is more valuable.

## Exercise 2

Based on the network we made in class, it is possible to a model that learns the XOR function. The XOR funtion is not linear seprable, so the only way to make network that works would be for us to violate the sequntial layer rule.

## Exercise 3

In [0]:
import tensorflow.keras
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import boston_housing
from sklearn.datasets import load_boston
import random
import math

In [0]:
(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

### A) Compute the dimensions of the data structures.

In [81]:
print( "There are", train_data.shape[0] + test_data.shape[0], "total records and",
      train_data.shape[1], "features in this dataset.")


There are 506 total records and 13 features in this dataset.


### B) Construct a suitable testing set, training set, and validation set for this data. 


In [0]:
def split_data(orig_train_data, orig_train_targets, orig_test_data, orig_test_targets):
  
  # convert np arrays to pandas df to combine them
  orig_train_df = pd.DataFrame(orig_train_data)
  orig_test_df = pd.DataFrame(orig_test_data)
  all_data_df = orig_train_df.append(orig_test_df)

  # confirm it worked
  target_shape = (orig_train_data.shape[0] + orig_test_data.shape[0], orig_train_data.shape[1])
  assert (all_data_df.shape == target_shape)

  # simularly combine targets
  all_targets = np.append(orig_train_targets, orig_test_targets)
  assert (len(all_targets) == target_shape[0])

  # split into training and testing data
  # 15% testing
  # 15% validation
  # 70% training
  all_indices = list(range(0, all_data_df.shape[0]))
  test_indeces= random.sample(range(0, all_data_df.shape[0]), 
                               math.floor(all_data_df.shape[0] *.15) )
  remaining_indices = np.setdiff1d(all_indices, test_indeces).tolist()
  val_indicies= random.sample(remaining_indices, 
                               math.floor(all_data_df.shape[0] *.15) )
  train_indices = np.setdiff1d(remaining_indices, val_indicies).tolist()
  
  #slit the data
  test_df = all_data_df.iloc[test_indeces]
  test_targets = all_targets[test_indeces]
  
  val_df = all_data_df.iloc[val_indicies]
  val_targets = all_targets[val_indicies]
  
  train_df = all_data_df.iloc[train_indices]
  train_targets = all_targets[train_indices] 

  #make sure it worked
  assert test_df.shape[0] == len(test_targets) == val_df.shape[0] == len(val_targets)
  assert test_df.shape[1] == train_df.shape[1] == val_df.shape[1] == 13
  assert (test_df.shape[0] + val_df.shape[0] + train_df.shape[0]) == all_data_df.shape[0]
  assert len(train_targets) +len(test_targets) + len(val_targets) == len(all_targets)

  # return
  return (train_df, train_targets, val_df, val_targets, test_df, test_targets)                      

### C) Create one new synthetic feature that could be useful for machine learning in this domain. Explain what it is and why it might be useful, and submit code to add it to the dataset.



A useful feature to add to this dataset could be taxes per $10,000 of rooms. This is another metric to indicate the cost of living in a nieghboorhood. This is epecially nice because it maps taxes relative to the size of the house.

In [123]:
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
full_df = train_df.append(test_df)
full_df.rename({0:'crim', 1: 'zn', 2:'indus', 
                3: 'chas', 4: 'nox', 6: 'age',
                5: 'rm', 7: 'dis', 
                8: 'rad', 9: 'tax', 
                10: 'ptratio', 11: 'black', 12: 'lstat'}, axis=1, inplace=True)

full_df['taxes_per_room'] = full_df['tax']/full_df['rm']
full_df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,tax_per_room
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72,49.983719
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11,45.729304
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26,134.004024
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01,37.104522
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65,104.454203
